<a href="https://colab.research.google.com/github/ginny0410/aop113b/blob/main/EX05_01_Echo_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install line-bot-sdk

In [2]:
!pip install pyngrok
!pip install flask_ngrok

In [3]:
!pip install flask

In [8]:

from google.colab import userdata

from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def initngrok():
  ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
  ngrok.connect(5000)
  run_with_ngrok(app)

from flask import Flask, request, abort
from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANELSECRET'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )

initngrok()
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://0144-34-127-103-181.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 09:16:36] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 09:16:37] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 09:16:58] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 09:17:13] "POST /callback HTTP/1.1" 200 -
